In [2]:
import pandas as pd
import math

## ReadingNames and renaming columns


In [9]:
Company_X_Orders  = pd.read_excel("Company X - Order Report.xlsx")
Company_X_Orders .head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [10]:
Company_Pincode = pd.read_excel("Company X - Pincode Zones.xlsx")
Company_Pincode.rename(columns = {'Zone': 'Delivery Zone as per X'},inplace=True)
Company_Pincode = Company_Pincode['Delivery Zone as per X']

In [11]:
Courier_company_invoice = pd.read_excel("Courier Company - Invoice.xlsx")
Courier_company_invoice.drop(['Warehouse Pincode','Customer Pincode'],axis=1,inplace=True)
Courier_company_invoice.columns  = ['AWB Number', 'Order ID', 'Total weight as per Courier Company (KG)','Delivery Zone charged by Courier Company', 'Type of Shipment', 'Charges Billed by Courier Company (Rs.) ']
Courier_company_invoice.head()

,AWB Number,Order ID,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.)
0,1091117222124,2001806232,1.30,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,d,Forward charges,45.4


In [12]:
SKU_Master = pd.read_excel("Company X - SKU Master.xlsx")
SKU_Master.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [13]:
Courier_company_Rates = pd.read_excel('Courier Company - Rates.xlsx')

## Calculating Total weight as per X (KG) and Weight slab as per X (KG)

In [14]:
df4 = pd.merge(Company_X_Orders ,SKU_Master,on='SKU')
df4['new_weight'] = df4["Order Qty"]* 0.001* df4["Weight (g)"]

In [15]:
df4

,ExternOrderNo,SKU,Order Qty,Weight (g),new_weight
0,2001827036,8904223818706,1.0,127,0.127
1,2001821995,8904223818706,1.0,127,0.127
2,2001819252,8904223818706,1.0,127,0.127
3,2001816996,8904223818706,1.0,127,0.127
4,2001814580,8904223818706,1.0,127,0.127
...,...,...,...,...,...
396,2001806616,8904223819123,1.0,250,0.250
397,2001806567,8904223815804,1.0,160,0.160
398,2001806567,8904223818577,1.0,150,0.150
399,2001806408,8904223819437,2.0,552,1.104


In [16]:
temp_df4 = df4[["ExternOrderNo","new_weight"]].groupby('ExternOrderNo').sum()
temp_df4.reset_index(level=0, inplace=True)
temp_df4

,ExternOrderNo,new_weight
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245
...,...,...
119,2001821995,0.477
120,2001822466,1.376
121,2001823564,0.672
122,2001825261,1.557


In [17]:
temp_df4['new_weight'] = temp_df4['new_weight'].round(1)
temp_df4

,ExternOrderNo,new_weight
0,2001806210,0.2
1,2001806226,0.5
2,2001806229,0.5
3,2001806232,1.3
4,2001806233,0.2
...,...,...
119,2001821995,0.5
120,2001822466,1.4
121,2001823564,0.7
122,2001825261,1.6


In [18]:
temp_df4['slab_weight'] = temp_df4['new_weight'].apply(lambda x: (math.ceil(x*2)/2) )

In [19]:
temp_df4.columns = ['Order ID','Total weight as per X (KG)','Weight slab as per X (KG)']
temp_df4

,Order ID,Total weight as per X (KG),Weight slab as per X (KG)
0,2001806210,0.2,0.5
1,2001806226,0.5,0.5
2,2001806229,0.5,0.5
3,2001806232,1.3,1.5
4,2001806233,0.2,0.5
...,...,...,...
119,2001821995,0.5,0.5
120,2001822466,1.4,1.5
121,2001823564,0.7,1.0
122,2001825261,1.6,2.0


In [20]:
temp_df4 = pd.merge(Courier_company_invoice,temp_df4,on='Order ID')

In [21]:
temp_df4 = pd.concat([temp_df4,Company_Pincode],axis=1)

In [22]:
c = []
for i,j,k in zip(temp_df4['Type of Shipment'],temp_df4['Delivery Zone as per X'],temp_df4['Weight slab as per X (KG)']):
    if i == 'Forward charges':
        if k > 0.5:
            c.append(round((Courier_company_Rates['fwd_'+j+'_fixed'][0]) + ((k-0.5)/0.5 * (Courier_company_Rates['fwd_'+j+'_additional'][0])),1))
        else:
            c.append(round(Courier_company_Rates['fwd_'+j+'_fixed'][0]))
    if i == 'Forward and RTO charges':
        if k> 0.5:
            c.append(round((Courier_company_Rates['rto_'+j+'_fixed'][0]) + ((k-0.5)/0.5 * (Courier_company_Rates['rto_'+j+'_additional'][0])),1))
        else:
            c.append(round(Courier_company_Rates['rto_'+j+'_fixed'][0]))
                     

In [23]:
Expected = pd.DataFrame(c)
Expected.columns = ['Expected Charge as per X (Rs.)']

In [32]:
Final_df = pd.concat([temp_df4,Expected],axis=1)
Final_df['Weight slab charged by Courier Company (KG)'] = Final_df['Total weight as per Courier Company (KG)'].apply(lambda x: (math.ceil(x*2)/2) )

In [36]:
Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'] = Final_df['Expected Charge as per X (Rs.)'] - Final_df['Charges Billed by Courier Company (Rs.)']

KeyError: 'Charges Billed by Courier Company (Rs.)'

In [28]:
col_rear = pd.read_excel('Expected_Result.xlsx',sheet_name='Calculations').columns
col_rear

Index(['Order ID', 'AWB Number', 'Total weight as per X (KG)',
       'Weight slab as per X (KG)', 'Total weight as per Courier Company (KG)',
       'Weight slab charged by Courier Company (KG)', 'Delivery Zone as per X',
       'Delivery Zone charged by Courier Company',
       'Expected Charge as per X (Rs.)',
       'Charges Billed by Courier Company (Rs.) ',
       'Difference Between Expected Charges and Billed Charges (Rs.)'],
      dtype='object')

In [37]:
Final_df = Final_df[col_rear]

KeyError: "['Difference Between Expected Charges and Billed Charges (Rs.)'] not in index"

In [30]:
Final_df

,AWB Number,Order ID,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Total weight as per X (KG),Weight slab as per X (KG),Delivery Zone as per X,Expected Charge as per X (Rs.),Weight slab charged by Courier Company (KG)
0,1091117222124,2001806232,1.30,d,Forward charges,135.0,1.3,1.5,d,135.0,1.5
1,1091117222194,2001806273,1.00,d,Forward charges,90.2,0.6,1.0,d,90.2,1.0
2,1091117222931,2001806408,2.50,d,Forward charges,224.6,2.3,2.5,d,224.6,2.5
3,1091117223244,2001806458,1.00,b,Forward charges,61.3,0.7,1.0,b,61.3,1.0
4,1091117229345,2001807012,0.15,d,Forward charges,45.4,0.2,0.5,d,45.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,d,Forward charges,90.2,0.5,0.5,b,33.0,1.0
120,1091117614452,2001809383,0.50,d,Forward and RTO charges,86.7,0.6,1.0,b,48.8,0.5
121,1091120922803,2001820978,0.50,d,Forward charges,45.4,0.5,0.5,b,33.0,0.5
122,1091121844806,2001811475,0.50,b,Forward charges,33.0,0.7,1.0,e,112.1,0.5


In [31]:
correct_count = Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)']==0].count()
correct_amt = Final_df['Charges Billed by Courier Company (Rs.) '][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)']==0].sum()
print(correct_count,correct_amt)

KeyError: 'Difference Between Expected Charges and Billed Charges (Rs.)'

In [38]:
Over_count = Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0].count()
Over_amt = Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)']< 0].sum()
print(Over_amt,Over_count)

KeyError: 'Difference Between Expected Charges and Billed Charges (Rs.)'

In [39]:
under_count = Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0].count()
under_amt = Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0].sum()
print(under_amt,under_count)

KeyError: 'Difference Between Expected Charges and Billed Charges (Rs.)'

In [40]:
dic = [[correct_count,correct_amt],[Over_count,Over_amt],[under_count,under_amt]]
Summary = pd.DataFrame(dic,index=['Total Orders - Correctly Charged','Total Orders - Over Charged','Total Orders - Under Charged'])

NameError: name 'correct_count' is not defined

In [41]:
Summary.columns = ['Count','Amount'] 

NameError: name 'Summary' is not defined

In [42]:
Summary

NameError: name 'Summary' is not defined

In [43]:
writer = pd.ExcelWriter('Final_result.xlsx',engine='xlsxwriter')   
Summary.to_excel(writer,sheet_name='Summary')
Final_df.to_excel(writer,sheet_name='Calculations',index=False)
writer.save()

ModuleNotFoundError: No module named 'xlsxwriter'